<a href="https://colab.research.google.com/github/jessiececilya/Projects/blob/main/XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score  #for accuracy_score
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.metrics import confusion_matrix #for confusion matrix

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
train= pd.read_csv("https://raw.githubusercontent.com/jessiececilya/ML-Data/main/WiDS/TrainingWiDS2021.csv")

In [ ]:
predict= pd.read_csv('https://raw.githubusercontent.com/jessiececilya/ML-Data/main/WiDS/UnlabeledWiDS2021.csv')

In [ ]:
trainx= train.drop(['diabetes_mellitus','Unnamed: 0','encounter_id','hospital_id','icu_id'],axis=1)

In [ ]:
trainy= train['diabetes_mellitus']

In [ ]:
trainy.head()

0    1
1    1
2    0
3    0
4    0
Name: diabetes_mellitus, dtype: int64

In [ ]:
cate= ['ethnicity',
 'gender',
 'hospital_admit_source',
 'icu_admit_source',
 'icu_stay_type',
 'icu_type']


In [ ]:
trainx= pd.get_dummies(trainx, columns= cate)
# #Label encoder
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# for ob in cate:
#     trainx[ob] = le.fit_transform(trainx[ob].astype(str))
#     predict[ob] = le.fit_transform(predict[ob].astype(str))


In [ ]:
trainx.columns

Index(['age', 'bmi', 'elective_surgery', 'height', 'pre_icu_los_days',
       'readmission_status', 'weight', 'albumin_apache', 'apache_2_diagnosis',
       'apache_3j_diagnosis',
       ...
       'icu_stay_type_readmit', 'icu_stay_type_transfer', 'icu_type_CCU-CTICU',
       'icu_type_CSICU', 'icu_type_CTICU', 'icu_type_Cardiac ICU',
       'icu_type_MICU', 'icu_type_Med-Surg ICU', 'icu_type_Neuro ICU',
       'icu_type_SICU'],
      dtype='object', length=209)

In [ ]:
pred= predict.drop(['Unnamed: 0','encounter_id','hospital_id','icu_id'],axis=1)

In [ ]:
# Save the entire model as a SavedModel.
# !mkdir -p saved_model
# modelf.save('saved_model/my_model')

In [ ]:
pred= pd.get_dummies(pred, columns= cate)

In [ ]:
set(pred.columns)-set(trainx.columns)


set()

In [ ]:
#pred= pd.get_dummies(pred, columns= cate)

In [ ]:
set(trainx.columns)- set(pred.columns)

set()

In [ ]:
addcols= ['hospital_admit_source_Acute Care/Floor', 'hospital_admit_source_ICU', 'hospital_admit_source_Observation', 'hospital_admit_source_Other', 'hospital_admit_source_PACU']
for e in addcols:
  pred[''+e]=0

In [ ]:
model= XGBClassifier()
params = {
    "learning_rate" :[0.1],
    "max_depth" : [5],
    "n_estimators" : [1000]
}

modelf = GridSearchCV(model,params, cv=2, scoring="accuracy", n_jobs= -1, verbose = 1)

x=modelf.fit(trainx,trainy)
output1=x.predict_proba(pred[trainx.columns])

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 88.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 88.3min finished


In [ ]:
model= XGBClassifier()
params = {
    "learning_rate" :[0.1],
    "max_depth" : [5],
    "n_estimators" : [2000]
}

modelf = GridSearchCV(model,params, cv=2, scoring="accuracy", n_jobs= -1, verbose = 1)

z= modelf.fit(trainx,trainy)
output=z.predict_proba(pred[trainx.columns])

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 34.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 34.6min finished


In [ ]:
z.best_params_

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}

In [ ]:
output=z.predict_proba(pred[trainx.columns])

*** M1: 0.1, 1000, dep 5 85.151; M2: 0.1 dep 5 1000 est 84.6 ***

In [ ]:
encounterid= predict['encounter_id']

out= pd.DataFrame({
    'encounter_id' : encounterid.tolist(),
    'diabetes_mellitus' : output[:,1].tolist()
    
})
out.sort_values('encounter_id', ascending=True,inplace=True)

In [ ]:
from google.colab import files
out.to_csv('1kest.csv',index=False)
files.download('1kest.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>